In [57]:
# LSTM
import pandas as pd
import numpy as np
from dataset import LABELS
train = pd.read_pickle('features_training.pkl')
val = pd.read_pickle('features_validation.pkl')

In [58]:
X_train = np.array([x[0] for x in train])
y_train = np.array([x[1] for x in train])
X_val = np.array([x[0] for x in val])
y_val = np.array([x[1] for x in val])

In [59]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
label_encoder = LabelEncoder()

# Fit the label encoder using your labels (combine both train and val labels if they have different classes)
label_encoder.fit(np.concatenate((y_train, y_val)))

# Transform your string labels to integer labels for both training and validation sets
y_train_encoded = label_encoder.transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

In [66]:
# prepare lstm model to classify the audio commends
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional, TimeDistributed, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

# define model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))

model.add(Dense(len(LABELS), activation='softmax'))
opt = Adam(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

# train model


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_13 (LSTM)              (None, 39, 128)           88576     
                                                                 
 dropout_17 (Dropout)        (None, 39, 128)           0         
                                                                 
 lstm_14 (LSTM)              (None, 39, 128)           131584    
                                                                 
 dropout_18 (Dropout)        (None, 39, 128)           0         
                                                                 
 lstm_15 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_19 (Dropout)        (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 128)              

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [97]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional, TimeDistributed, BatchNormalization

results = pd.DataFrame(columns=['lstm_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])

# Set your model's hyperparameters
for lstm_units in [64, 128, 256]:
    for dropout_rate in [0.2, 0.4, 0.6]:
            for epoch in [10, 20, 30, 40]:
                  for batch in [32, 64, 128]:

                        num_classes = len(y_train_encoded)  # Number of unique classes in your dataset

                        input_shape = (39, 44)

                        model = Sequential([
                            Bidirectional(LSTM(lstm_units, return_sequences=True), input_shape=input_shape),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            Bidirectional(LSTM(lstm_units, return_sequences=True)),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            LSTM(lstm_units, return_sequences=True),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            TimeDistributed(Dense(64, activation='relu')),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            TimeDistributed(Dense(32, activation='relu')),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            LSTM(lstm_units),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            Dense(num_classes, activation='softmax')
                        ])


                        # Compile the model
                        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                        # Train the model with your training set and validate it with your validation set
                        epochs = epoch
                        batch_size = batch
                        history = model.fit(X_train, y_train_encoded, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val_encoded))

                        results = np.concatenate((results, pd.DataFrame([[lstm_units, dropout_rate, epoch, batch, history.history['loss'][-1], history.history['loss'], history.history['accuracy'][-1], history.history['accuracy'], history.history['val_loss'][-1], history.history['val_loss'], history.history['val_accuracy'][-1], history.history['val_accuracy']]], 
                                                                        columns=['lstm_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])), axis=0)


Epoch 1/10
214/214 [==============================] - 24s 79ms/step - loss: 7.0679 - accuracy: 0.0974 - val_loss: 4.0350 - val_accuracy: 0.1478
Epoch 2/10
214/214 [==============================] - 16s 74ms/step - loss: 2.9956 - accuracy: 0.2135 - val_loss: 2.5939 - val_accuracy: 0.2307
Epoch 3/10
214/214 [==============================] - 16s 75ms/step - loss: 2.3891 - accuracy: 0.2976 - val_loss: 2.3668 - val_accuracy: 0.3070
Epoch 4/10
214/214 [==============================] - 16s 77ms/step - loss: 2.1566 - accuracy: 0.3582 - val_loss: 2.2170 - val_accuracy: 0.3557
Epoch 5/10
214/214 [==============================] - 16s 77ms/step - loss: 1.9021 - accuracy: 0.4253 - val_loss: 2.1590 - val_accuracy: 0.3886
Epoch 6/10
214/214 [==============================] - 17s 82ms/step - loss: 1.7554 - accuracy: 0.4759 - val_loss: 2.0464 - val_accuracy: 0.4235
Epoch 7/10
214/214 [==============================] - 17s 78ms/step - loss: 1.5497 - accuracy: 0.5342 - val_loss: 2.0235 - val_accuracy:

In [ ]:
results.to_pickle('model_lstm')

array([[64, 0.2, 2, 32, 3.100442886352539,
        list([6.995808124542236, 3.100442886352539]),
        0.16634966433048248, 0.16634966433048248, 2.812389373779297,
        3.762976884841919, 0.18740805983543396, 0.18740805983543396]],
      dtype=object)

In [80]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])